# `LibreASR`

## Imports

In [ ]:
%reload_ext autoreload
%autoreload 3
# %matplotlib inline

In [ ]:
from libreasr.lib.imports import *

## `Config`

In [ ]:
# cfgs = ["./config/base.yaml", "./config/contrastive-stft.yaml"]
cfgs = ["./config/base.yaml"]
conf, lang, builder_train, builder_valid, db, m, learn = parse_and_apply_config(path=cfgs)
(conf, lang, db, m, learn)

In [ ]:
builder_train.print()
builder_valid.print()

In [ ]:
builder_train.plot()

In [ ]:
tpl = db.one_batch()
X, Ym, _, _ = tpl[0]
Y, Y_lens, X_lens = tpl[1]
what(X), what(X_lens), what(Y), what(Y_lens)

In [ ]:
# X_lens, Y_lens
# m(cudaize(tpl[0])).shape

### `Augmentation`

In [ ]:
# db.aug(n=3000, benchmark=True)

In [ ]:
# import matplotlib as mpl
# mpl.rcParams['figure.dpi'] = 400
# db.aug(n=5, benchmark=False, only=-3) #0: signal, 7: spectro

In [ ]:
# db.aug(n=5)#, only=0)

## `Model`

In [ ]:
device = conf["cuda"]["device"]
m = m.to(device)

In [ ]:
print(m)
print("Encoder   n_params:", n_params(m.encoder))
# print("Predictor n_params:", n_params(m.predictor))

Timing

In [ ]:
%%time
_in = db.one_batch()[0]

In [ ]:
%%time
m.eval()
with torch.no_grad():
    _out = m(cudaize(_in, device), return_loss=False)
    print("X", _in[0].shape, "->", _out.shape)
    print("Y", _in[1].shape)

## `Train`

### `Load from Checkpoint`

In [ ]:
load = False # TRUE   ::::::::::::::::::::::::::::::::::::::::::::::::!!!!!!!!!!!! ! ! ! !
load_with_opt = True
load_strict = True
load_encoder = False
load_encoder_file = "models/TRAINED/multi-contrastive-11ep-0.159val-wandb-zw7fv16h.pth"
print_tb = True

In [ ]:
ckpt = "best_wer"
ckpt = "libreasr-de-24-12-2020"
ckpt = "model"
ckpt_load = "model"

In [ ]:
print("Before Load:", m.encoder.input_norm.weight[0].item())

if load:
    ns = learn.model
    try:
        if conf["training"]["noisystudent"]:
            learn.model = ns.teacher[0]
            learn.load(ckpt_load, with_opt=False, strict=False)
            learn.model = ns
            learn.create_opt()
        else:
            learn.load(ckpt_load, with_opt=load_with_opt, strict=load_strict) # OPT TRUE
    except Exception as e:
        print("failed loading ckpt", ckpt)
        print(e)
    finally:
        if conf["training"]["noisystudent"]:
            learn.model = ns
            learn.create_opt()
if load_encoder:
    s = torch.load(load_encoder_file)
    sm = s['model']
    # sm = s
    d = sm
    
    # fix preprocessor / spec
    n = {}
    for k in d.keys():
        if k.startswith("spec"):
            n["preprocessor." + k] = d[k]
    d.update(n)
    
    # filter out predictor & spec
    print("raw:", list(d.keys()))
    d = {k: v for k, v in d.items() if not k.startswith("latents")}
    d = {k: v for k, v in d.items() if not k.startswith("temperature")}
    d = {k: v for k, v in d.items() if not k.startswith("predictor")}
    # d = {k: v for k, v in d.items() if not k.startswith("preprocessor")}
    # d = {k: v for k, v in d.items() if not k.startswith("encoder.input_norm")}
    # d = {k: v for k, v in d.items() if not k.startswith("encoder.rnn_stack.rnns.0.weight_ih_l0")}
    # d = {k: v for k, v in d.items() if not k.startswith("joint")}
    print("filtered:", list(d.keys()))
    
    m.load_state_dict(d, strict=False)
    
print("After  Load:", m.encoder.input_norm.weight[0].item())

In [ ]:
# save for later
# learn.save(load_encoder_file.replace(".pth", "-pretrained").replace("models/", ""), with_opt=False)

In [ ]:
# check forward and backward
# m(cudaize(_in)).sum().backward()

In [ ]:
# learn.save("base", with_opt=False)

In [ ]:
# %%time
# m.eval()
# with torch.no_grad():
#     if conf["training"]["noisystudent"]:
#         _out = m.teacher[0].transcribe_batch(cudaize(_in))
#     else:
#         _out = m.transcribe_batch(cudaize(_in))
# _ = [print(x) for x in _out]

In [ ]:
# %%time
# m.train()
# _out = m(cudaize(_in))
# _out.shape

### `DistributedDataParallel`

In [ ]:
# setup if necessary
ddp = conf.get("training", {}).get("ddp", {}).get("enable", False)
ctx = contextlib.suppress
if ddp:
    from fastai.distributed import DistributedDL
    
    # fix -1s (required for sending around idxs)
    @patch
    def get_idxs(self: DistributedDL):
        idxs = self.dl.get_idxs()       # compute get_idxs in all ranks (we'll only use rank 0 but size must be consistent)
        idxs = self._broadcast(idxs,0)  # broadcast and receive it from rank 0 to all
        # filter out -1s introduced in DynamicBucketingDL
        if isinstance(idxs[0], (list, tuple)):
            idxs = list(map(lambda s: list(filter(lambda x: x != -1, s)), idxs))
        self.n = len(idxs)              # we assumed n was dl.n but we really care about number of idxs
        # add extra samples to make it evenly divisible
        self.n_padded = DistributedDL._round_to_multiple(self.n,self.world_size)
        idxs += (idxs * (self.n_padded//self.n))[:self.n_padded-self.n] # idx needs to be repeated when n_padded>>n
        # slice padded idxs so that each rank gets self.n_padded//self.world_size tensors
        return idxs[self.rank*self.n_padded//self.world_size:(self.rank+1)*self.n_padded//self.world_size]
    
    dev = torch.device("cuda:" + str(rank_distrib()))
    learn.remove_cb(DeviceCallback)
    learn.add_cb(DeviceCallback(dev))
    ctx = learn.distrib_ctx
    print("DistributedDataParallel activated")

### `fit`

In [ ]:
learn.unfreeze()

In [ ]:
# learn.lr_find(num_it=400)
# learn.lr_find(num_it=200)

In [ ]:
# hypers
do_warmup = True
do_tuning = True
do_cyclical = False
epochs = 50
warmup_epochs = 3.9
warmup = int(warmup_epochs + 1)

# initial fit: 5e-4 / (1e-3 for large batch)
# cat runs:    3e-4
lr = 1e-3 # 5e-4 # 3e-4 # 5e-4 # 7e-4 # 5e-4 # 3e-4
wd = 0.0 # 0.01

div = 500. # 100.
div_final = 1.001

pct_start = warmup_epochs / warmup
device = conf["cuda"]["device"]

# cyclical stage
cyc_epochs = 3
cyc_b = 1
cyc_lr = 5e-5

In [ ]:
def fit_safe(func, *args, **kwargs):
    try:
        func(*args, **kwargs)
    except:
        print(learn.recorder.values)
        import traceback
        traceback.print_exc()
        
def fix_opt():
    learn.save("tmp", with_opt=True)
    learn.load("tmp", with_opt=True, device=device)

In [ ]:
with ctx():
    # WARMUP
    print(learn.cbs)
    if do_warmup:
        print("STAGE: WARMUP")
        fit_safe(learn.fit_one_cycle, warmup, lr, wd=wd, cbs=learn.extra_cbs, div_final=div_final, div=div, pct_start=pct_start, moms=(0.95,)*3)
        fix_opt()

    # TUNING
    if do_tuning:
        print("STAGE: TUNING")
        fit_safe(learn.fit, epochs, lr, wd=wd, cbs=learn.extra_cbs)
        fix_opt()
        
    # CYCLICAL
    if do_cyclical:
        print("STAGE: CYCLICAL")
        fit_safe(learn.fit_sgdr, cyc_epochs, cyc_b, cyc_lr, wd=wd, cbs=learn.extra_cbs)
        fix_opt()

In [ ]:
raise Exception("done")

### `checks`

In [ ]:
learn.recorder.values

In [ ]:
# save model for later
# learn.save(checkpoints[-1], with_opt=True)
learn.save(ckpt, with_opt=False)

In [ ]:
learn.save("multi-pretrained", with_opt=False)

## `Quantization`

In [ ]:
from libreasr.lib.quantization import quantize_rnnt, save_quantized, load_quantized

# quantize
m8 = quantize_rnnt(m, {Encoder, Predictor, Joint})

# save
save_quantized(m8)

# load
m9 = load_quantized(m, lang)
m9.lang = lang

In [ ]:
def infer(mod):
    # mod = mod.eval()
    with torch.no_grad():
        q = mod.transcribe_batch(_in)
    return q

In [ ]:
%%time
infer(m9)

In [ ]:
learn.model = m9
# learn.model = m

## `Decode`

In [ ]:
sampls = 2000 # 99999999
test_name = "dev-cv" # "dev-all-cat-mix-cont"

In [ ]:
_ = list(eval_speech_model(learn, min_samples=sampls, train=False, save_best=False, pcent=1.0)) # , device="cpu"))

In [ ]:
ylens = map(lambda x: x.get("text/ground_truth"), _)
ylens = filter(lambda x: x is not None and len(x) != 0, ylens)
ylens = map(lambda x: len(x), ylens)
ylens = np.array(list(ylens))

wers = map(lambda x: x.get("metrics/wer", None), _)
wers = filter(lambda x: x is not None, wers)
wers = np.array(list(wers))
ylens[:3], wers[:3], len(ylens), len(wers)

In [ ]:
stack = np.stack([ylens, wers]).T
stack.shape

In [ ]:
df = pd.DataFrame(stack, columns=["ylen", "wer"])
df.head()

In [ ]:
mean_wer = df.groupby("ylen").mean()["wer"].values
std_wer = df.groupby("ylen").std()["wer"].values
border_upper = mean_wer + std_wer / 2
border_lower = mean_wer - std_wer / 2
count = df.groupby("ylen").count()["wer"].values

In [ ]:
fig = plt.figure()
plot_wer = fig.add_subplot(111)
plot_count = plot_wer.twinx()
x = np.arange(len(mean_wer))

# wer
plot_wer.plot(mean_wer)
plot_wer.set_ylim(0.0, 1.2)
plot_wer.set_ylabel("WER")
plot_wer.set_xlabel("Label Length")

# std
plot_wer.fill_between(x, border_upper, border_lower, alpha=0.2)

# count
plot_count.bar(x, count, alpha=0.1)
plot_count.set_ylabel("Number of Examples")
plot_count.set_yscale("log")

# save plot
# plt.savefig(f"{test_name}.png", dpi=200)

In [ ]:
# save tests
import pickle
pickle.dump(_, open(f'{test_name}.pkl', 'wb'))

In [ ]:
_[:2]

In [ ]:
def _sort(results, rev):
    results = filter(lambda x: "text/ground_truth" in list(x.keys()), results)
    results = sorted(results, key=lambda x: x["metrics/wer"], reverse=rev)
    return results

def best(results):
    return _sort(results, rev=False)

def worst(results, n=25):
    results = _sort(results, rev=True)[:n]
    df = builder_valid.df
    for res in results:
        def filter_fn(row):
            s1 = (res["text/ground_truth"])
            s2 = sanitize_str(row.label.decode("utf-8"))
            in_ok = s1 in s2
            l_ok = abs(len(s1) - len(s2)) < 10
            return in_ok and l_ok
        rows = df[df.apply(filter_fn, axis=1)]
        row = rows
        print(len(row))
        if len(row) >= 1:
            row = row.iloc[0]
            file = row.file
            sr = int(row.sr)
            xstart = int(row.xstart / 1000. * sr)
            xlen = int(row.xlen / 1000. * sr)
            data, sr = torchaudio.load(file, frame_offset=xstart, num_frames=xlen)
            print("PRED:", res["text/prediction"])
            print("TRUE:", res["text/ground_truth"])
            print(f"WER:  {res['metrics/wer'] * 100.:.2f}%")
            display(Audio(data.numpy(), rate=sr))
        else: print("err")

In [ ]:
worst(_, n=10)

In [ ]:
m.joint.joint[2].bias

In [ ]:
m.joint.joint[2].bias

In [ ]:
learn.save("it-best-noopt", with_opt=False)

# Contrastive Analysis

In [ ]:
batches = []
for batch, _ in zip(db.valid, range(2)):
    batches.append(batch)

bv = batches[0][0]
bv[0].shape, bv[1].shape

In [ ]:
# set some values to zero
# to check if similarity is still okish
X, Y = bv[0].size(1), bv[1].size(1)
bv[0][:, :X // 10] = 0
bv[1][:, :Y // 10] = 0

In [ ]:
class ContrastiveAnalyzer:
    def __init__(self, model):
        self.model = model
        self.batch = None
        self.outputs = None
        self.text = None
        self.mat = None
    
    def rep(self, b, plot=False, save=False, dpi=300):
        self.batch = b
        m = self.model
        m.eval()
        with torch.no_grad():
            e, p, a, labels = m(cudaize(b), return_logits=True)
            self.outputs = (e, p, a, labels)
            print("Loss:", nn.CrossEntropyLoss()(a, labels).item())
        if plot:
            import matplotlib as mpl
            mpl.rcParams['figure.dpi'] = dpi
            plt.imshow(a.cpu().numpy())
            plt.colorbar()
            plt.ylabel("Transcript")
            plt.xlabel("Audio")
            plt.title(f"Representation Similarity")
            if bool(save):
                plt.savefig(save, dpi=300)
                
    def metrics(self, learn):
        from fastai2.metrics import accuracy
        accs1 = []
        accs2 = []
        learn.model.eval()
        from tqdm import tqdm
        for b in tqdm(learn.dls.valid):
            with torch.no_grad():
                inp, targ = m(cudaize(b[0]))
            a1 = accuracy(inp, targ, axis=0).item()
            a2 = accuracy(inp, targ, axis=1).item()
            accs1.append(a1)
            accs2.append(a2)
        return np.array(accs1).mean(), np.array(accs2).mean()
            
    def texts(self):
        # save
        l = []
        for i, b in enumerate(self.batch[1]):
            l.append([i, lang.denumericalize(b.numpy().tolist())])
        self.text = l
            
    def rank(self):
        mat = []
        l = self.text
        from difflib import SequenceMatcher
        for i, a in l:
            for j, b in l:
                if i != j:
                    ratio = SequenceMatcher(None, a, b).ratio()
                    mat.append([i, j, a, b, ratio])
        # sort
        mat = sorted(mat, key=lambda x: x[-1], reverse=True)
        self.mat = mat
        return mat
    
    def similar(self, n=10):
        return self.mat[:n]
    
    def circles(self):
        self.circles = list(filter(lambda x: x[-1] != 1., self.mat))[:10]
        for k in self.circles:
            print(k[0], k[1])
            
    def compare(self, p=True, thresh=0.75):
        p = self.outputs[1 if p else 0]
        sim2 = torch.einsum("i d, j d -> i j", p, p)
        sim2[sim2 < thresh] = 0.
        plt.imshow(sim2.cpu().numpy())
        plt.colorbar()
        plt.tick_params(bottom=True)
        from matplotlib.ticker import MultipleLocator
        ml = MultipleLocator(2)
        plt.axes().yaxis.set_minor_locator(ml)
        plt.axes().xaxis.set_minor_locator(ml)
        
    def activations(self, p=True, to=100):
        act = self.outputs[1 if p else 0]
        plt.imshow(act[:, :to].cpu().numpy())
        plt.colorbar()
        plt.tick_params(bottom=True)

In [ ]:
c = ContrastiveAnalyzer(m)

In [ ]:
_ = c.rep(bv)
_ = c.texts()

In [ ]:
# c.rep(bv, plot=True, dpi=300)

In [ ]:
# c.compare(p=True, thresh=0.9)

In [ ]:
# c.activations(p=False, to=100)

In [ ]:
# _ = c.rank()
# c.similar()

In [ ]:
c.metrics(learn)